In [22]:
import pandas as pd 
import glob
import os 
#this block of code takes the csv generated from Flight_ETL notebook
# And then grabs the latest one that was saved to the file path
data= "flight_data/raw/"
csv_files = glob.glob(os.path.join(data, "flight_arrivals_*.csv"))

latest_file = max(csv_files,key=os.path.getmtime)

flight_df = pd.read_csv(latest_file)
flight_df.head()

,flight_date,weekday,arrival_iataCode,arrival_terminal,arrival_scheduledTime,departure_iataCode,airline_name,flight_iataNumber,aircraft_modelText
0,2025-07-30,3,mci,NaN,23:34,phx,alphasky,as4285,boeing 737 max 8
1,2025-07-30,3,mci,NaN,23:34,phx,american airlines,aa623,boeing 737 max 8
2,2025-07-30,3,mci,NaN,09:25,atl,latam airlines,la6120,boeing 737-932(er)
3,2025-07-30,3,mci,NaN,09:25,atl,westjet,ws7243,boeing 737-932(er)
4,2025-07-30,3,mci,NaN,09:25,atl,delta air lines,dl2471,boeing 737-932(er)


In [23]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754 entries, 0 to 1753
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   flight_date            1754 non-null   object
 1   weekday                1754 non-null   int64 
 2   arrival_iataCode       1754 non-null   object
 3   arrival_terminal       39 non-null     object
 4   arrival_scheduledTime  1739 non-null   object
 5   departure_iataCode     1754 non-null   object
 6   airline_name           1754 non-null   object
 7   flight_iataNumber      1754 non-null   object
 8   aircraft_modelText     1720 non-null   object
dtypes: int64(1), object(8)
memory usage: 123.5+ KB


In [24]:
#A quick view of different airplane to see what we are work with
sorted(flight_df['airline_name'].dropna().unique())

['aer lingus',
 'aeromexico',
 'air canada',
 'air france',
 'air new zealand',
 'air tahiti nui',
 'alaska airlines',
 'alaska horizon',
 'allegiant air',
 'allegiant air (insomniac livery)',
 'alphasky',
 'amazon air',
 'american airlines',
 'american eagle',
 'ana',
 'british airways',
 'brussels airlines',
 'china eastern airlines',
 'copa airlines',
 'delta air lines',
 'delta connection',
 'dhl',
 'fedex',
 'finnair',
 'frontier (rio the jaguarundi livery)',
 'frontier (sombra the jaguar livery)',
 'frontier (tex the armadillo livery)',
 'hawaiian airlines',
 'iberia',
 'jet linx aviation',
 'kenya airways',
 'klm',
 'korean air',
 'latam airlines',
 'lufthansa',
 'malaysia airlines',
 'qantas',
 'qatar airways',
 'royal air maroc',
 'royal jordanian',
 'sa avianca',
 'sas',
 'southwest airlines',
 'southwest airlines (arizona one livery)',
 'southwest airlines (freedom one livery)',
 'southwest airlines (imua one livery)',
 'southwest airlines (louisiana one livery)',
 'southwes

In [25]:
# there's 3 different options domestic, international, and cargo we won't need cargo to predict demand for uber
domestic_airlines = [
    'alaska airlines', 'alaska horizon', 'allegiant air', 'american airlines',
    'delta air lines', 'delta connection', 'frontier (flint the wood stork livery)',
    'hawaiian airlines', 'jet linx aviation', 'southwest airlines',
    'spirit airlines', 'united airlines', 'united express'
]

international_airlines = [
    'aeromexico', 'air canada', 'air france', 'air new zealand', 'air tahiti nui',
    'ana', 'british airways', 'brussels airlines', 'china eastern airlines',
    'copa airlines', 'emirates', 'finnair', 'iberia', 'kenya airways', 'klm',
    'korean air', 'latam airlines', 'lufthansa', 'malaysia airlines', 'qantas',
    'qatar airways', 'sa avianca', 'sas', 'turkish airlines', 'virgin atlantic',
    'volaris', 'westjet'
]

cargo_airlines = ['amazon air', 'dhl', 'fedex', 'ups', 'alphasky']

In [26]:
# this function takes the list that was above and classifys if an air line is domestic, international or Cargo
def classify_airline(airline):
    airline = airline.lower()
    if airline in domestic_airlines:
        return 'Domestic'
    elif airline in international_airlines:
        return 'International'
    elif airline in cargo_airlines:
        return 'Cargo'
    else:
        return 'Unknown'

flight_df['airline_type'] = flight_df['airline_name'].apply(classify_airline)
flight_df.head()

,flight_date,weekday,arrival_iataCode,arrival_terminal,arrival_scheduledTime,departure_iataCode,airline_name,flight_iataNumber,aircraft_modelText,airline_type
0,2025-07-30,3,mci,NaN,23:34,phx,alphasky,as4285,boeing 737 max 8,Cargo
1,2025-07-30,3,mci,NaN,23:34,phx,american airlines,aa623,boeing 737 max 8,Domestic
2,2025-07-30,3,mci,NaN,09:25,atl,latam airlines,la6120,boeing 737-932(er),International
3,2025-07-30,3,mci,NaN,09:25,atl,westjet,ws7243,boeing 737-932(er),International
4,2025-07-30,3,mci,NaN,09:25,atl,delta air lines,dl2471,boeing 737-932(er),Domestic


In [27]:
#then we get rid of cargo for a new dataframe
flight_df_clean = flight_df[flight_df['airline_type'] != 'Cargo']
flight_df_clean = flight_df_clean.dropna(subset=['arrival_scheduledTime'])
flight_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1635 entries, 1 to 1753
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   flight_date            1635 non-null   object
 1   weekday                1635 non-null   int64 
 2   arrival_iataCode       1635 non-null   object
 3   arrival_terminal       39 non-null     object
 4   arrival_scheduledTime  1635 non-null   object
 5   departure_iataCode     1635 non-null   object
 6   airline_name           1635 non-null   object
 7   flight_iataNumber      1635 non-null   object
 8   aircraft_modelText     1602 non-null   object
 9   airline_type           1635 non-null   object
dtypes: int64(1), object(9)
memory usage: 140.5+ KB


In [28]:
#this is tranforming the schedule landing time into minutes and then into an number so that it can be run through the algorithm
flight_df_clean = flight_df_clean.copy()

flight_df_clean['arrival_time'] = pd.to_datetime(flight_df_clean['arrival_scheduledTime'],format='%H:%M', errors='coerce')
flight_df_clean['arrival_minutes'] = (flight_df_clean['arrival_time'].dt.hour * 60 + flight_df_clean['arrival_time'].dt.minute)

In [29]:
#Here we create another function, this is going to a sign a cluster based of density 
from sklearn.cluster import MeanShift

def run_mean_shift(flight_df_clean, bandwidth, label):
    clustered = []

    for date, group in flight_df_clean.groupby('flight_date'):
        group = group.copy()
        X = group[['arrival_minutes']].values

        if len(X) < 2:
            group[f'cluster_{label}'] = -1
        else:
            ms = MeanShift(bandwidth=bandwidth)
            group[f'cluster_{label}'] = ms.fit_predict(X)

        clustered.append(group)

    return pd.concat(clustered).reset_index(drop=True)

In [30]:
# we run two different cluster window's one with 90 minutes and the other 
df = run_mean_shift(flight_df_clean, bandwidth=90, label='90min')
df = run_mean_shift(df, bandwidth=180, label='180min')

In [31]:
df.head()

,flight_date,weekday,arrival_iataCode,arrival_terminal,arrival_scheduledTime,departure_iataCode,airline_name,flight_iataNumber,aircraft_modelText,airline_type,arrival_time,arrival_minutes,cluster_90min,cluster_180min
0,2025-07-30,3,mci,NaN,23:34,phx,american airlines,aa623,boeing 737 max 8,Domestic,1900-01-01 23:34:00,1414,1,1
1,2025-07-30,3,mci,NaN,09:25,atl,latam airlines,la6120,boeing 737-932(er),International,1900-01-01 09:25:00,565,5,2
2,2025-07-30,3,mci,NaN,09:25,atl,westjet,ws7243,boeing 737-932(er),International,1900-01-01 09:25:00,565,5,2
3,2025-07-30,3,mci,NaN,09:25,atl,delta air lines,dl2471,boeing 737-932(er),Domestic,1900-01-01 09:25:00,565,5,2
4,2025-07-30,3,mci,NaN,10:15,clt,american airlines,aa1616,airbus a320-214,Domestic,1900-01-01 10:15:00,615,5,2


In [32]:
def summarize_clusters(df, cluster_col, top_n=3, source='flight'):
    # Rename for consistency
    df = df.copy()
    df = df.rename(columns={cluster_col: 'cluster_id'})

    # Group and summarize
    summary = df.groupby(['flight_date', 'cluster_id','arrival_iataCode']).agg(
        start_min=('arrival_minutes', 'min'),
        end_min=('arrival_minutes', 'max'),
        flight_count=('arrival_minutes', 'count')
    ).reset_index()

    # Add readable times
    summary['start_time'] = pd.to_datetime(summary['start_min'], unit='m').dt.strftime('%H:%M')
    summary['end_time'] = pd.to_datetime(summary['end_min'], unit='m').dt.strftime('%H:%M')
    summary['duration_min'] = summary['end_min'] - summary['start_min']
    summary['midpoint_min'] = (summary['start_min'] + summary['end_min']) // 2
    summary['midpoint_time'] = pd.to_datetime(summary['midpoint_min'], unit='m').dt.strftime('%H:%M')

    # Rank and filter
    summary['rank'] = summary.groupby('flight_date')['flight_count'].rank(ascending=False, method='first')
    top_clusters = summary[summary['rank'] <= top_n].copy()
    top_clusters['source'] = source

    return top_clusters

In [ ]:
#taking the summarization funcation and applying it to the dataframe
top_90 = summarize_clusters(df, cluster_col='cluster_90min', top_n=3, source='flight_90min')
top_180 = summarize_clusters(df, cluster_col='cluster_180min', top_n=3, source='flight_180min')

In [ ]:
#then we take the to clusters dataframe the we summarized and we append the data
all_top_clusters = pd.concat([top_90, top_180], ignore_index=True)
all_top_clusters.head()

,flight_date,cluster_id,arrival_iataCode,start_min,end_min,flight_count,start_time,end_time,duration_min,midpoint_min,midpoint_time,rank,source
0,2025-07-30,0,mci,925,1062,46,15:25,17:42,137,993,16:33,2.0,flight_90min
1,2025-07-30,1,mci,1291,1439,48,21:31,23:59,148,1365,22:45,1.0,flight_90min
2,2025-07-30,4,mci,1095,1275,36,18:15,21:15,180,1185,19:45,3.0,flight_90min
3,2025-07-31,2,mci,815,995,50,13:35,16:35,180,905,15:05,2.0,flight_90min
4,2025-07-31,3,mci,1360,1435,43,22:40,23:55,75,1397,23:17,3.0,flight_90min


In [35]:
all_top_clusters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   flight_date       42 non-null     object 
 1   cluster_id        42 non-null     int64  
 2   arrival_iataCode  42 non-null     object 
 3   start_min         42 non-null     int32  
 4   end_min           42 non-null     int32  
 5   flight_count      42 non-null     int64  
 6   start_time        42 non-null     object 
 7   end_time          42 non-null     object 
 8   duration_min      42 non-null     int32  
 9   midpoint_min      42 non-null     int32  
 10  midpoint_time     42 non-null     object 
 11  rank              42 non-null     float64
 12  source            42 non-null     object 
dtypes: float64(1), int32(4), int64(2), object(6)
memory usage: 3.7+ KB


In [ ]:
# we then write the data to a csv so it can be later ingested into a dashboard
from datetime import datetime, timedelta 
timestamp = datetime.now().strftime('%Y%m%d')
raw_path = f"flight_data/modeled/Top_clusters{timestamp}.csv"
all_top_clusters.to_csv(raw_path, index=False)